In [1]:
import pandas as pd
import numpy as np

# We split the data for cross-validation

In [45]:
train_data = pd.read_csv('train_1_DEC.csv', index_col=0)

test_data = pd.read_csv('test_1_DEC.csv', index_col=0)

del train_data['Customer_ID']
del test_data['Customer_ID']

In [46]:
print(test_data.shape)
print(train_data.shape)

(30000, 32)
(70000, 33)


In [3]:
train_data

NameError: ignored

In [ ]:
test_data

In [31]:
column_names = train_data.columns.tolist()
print(column_names)

['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance', 'payday_loan', 'mortgage_loan', 'student_loan', 'not_specified', 'home_equity_loan', 'credit.builder_loan', 'No_Data', 'personal_loan', 'auto_loan', 'debt_consolidation_loan', 'EncodedSpent', 'EncodedValue_Payments', 'Credit_Mix_Encoded', 'Occupation_Encoded', 'Encoded_Payment', 'Credit_Score_Encoded']


In [47]:
categorical_columns = ['payday_loan', 'mortgage_loan', 'student_loan', 'not_specified',
 'home_equity_loan', 'credit.builder_loan', 'No_Data', 'personal_loan',
 'auto_loan', 'debt_consolidation_loan', 'EncodedSpent', 'EncodedValue_Payments',
 'Credit_Mix_Encoded', 'Occupation_Encoded', 'Encoded_Payment']

numerical_columns = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date',
 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries',
 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']

label_to_predict = 'Credit_Score'
#the following list is ordered by FOCI
selected_columns = [ "Outstanding_Debt", "Interest_Rate", "Num_Credit_Card", "Num_of_Loan", "Credit_History_Age", "Annual_Income", "Amount_invested_monthly", "Total_EMI_per_month", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Changed_Credit_Limit","Num_of_Delayed_Payment","Delay_from_due_date","Num_Credit_Inquiries", "Credit_Utilization_Ratio"]

# Method provided

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [50]:
label_encoder = LabelEncoder()
X_train = train_data.iloc[:, :-1]
X_test  = test_data
#y_train = train_data.iloc[:,32]
y_train = label_encoder.fit_transform(train_data[label_to_predict]).astype(int)

In [51]:
# Modeling and evaluation
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (
    BaggingClassifier,
    ExtraTreesClassifier,
    RandomForestClassifier,
    StackingClassifier,
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import joblib

In [52]:
bagging = BaggingClassifier(n_jobs=-1)
extraTrees = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
randomForest = RandomForestClassifier(n_jobs=-1)
histGradientBoosting = HistGradientBoostingClassifier()
XGB = XGBClassifier(n_jobs=-1)

model = StackingClassifier([
    ('bagging', bagging),
    ('extraTress', extraTrees),
    ('randomforest', randomForest),
    ('histGradientBoosting', histGradientBoosting),
    ('XGB', XGB)
], n_jobs=-1)

In [53]:
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('bagging', BaggingClassifier(n_jobs=-1)),
                               ('extraTress',
                                ExtraTreesClassifier(max_depth=10, n_jobs=-1)),
                               ('randomforest',
                                RandomForestClassifier(n_jobs=-1)),
                               ('histGradientBoosting',
                                HistGradientBoostingClassifier()),
                               ('XGB',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None...
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=-1,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   n_jobs=-1)

In [54]:
from sklearn.metrics import accuracy_score

In [55]:
print("Train Score: ",model.score(X_train, y_train))

Train Score:  0.9990857142857142


In [56]:
def generate_submission():
    list_of_predictions = model.predict(X_test)
    preds = label_encoder.inverse_transform(list_of_predictions)
    df = pd.DataFrame({'Credit_Score': preds}, index=X_test.index)
    df.to_csv('sandbox_submission.csv')

In [58]:
generate_submission()